# Benchmarking MongoDB

In this notebook we run some benchmarks with MongoDB.

In [ ]:
# Ensure MongoDB is running
! cd .. && docker compose up

In [ ]:
# Ensure the database is empty

In [ ]:
# Create collections and indexes

In [ ]:
# Load ZTF alerts into the database

In [ ]:
# Load NED alerts into the database

In [ ]:
# Run cross-matching

In [ ]:
# Run through some filters

In [ ]:
# Shut down Docker containers
! cd .. && docker compose down